In [11]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [12]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [13]:

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [14]:
from pyspark.sql import SparkSession

In [15]:
spark = SparkSession.builder.getOrCreate()


In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, lower, regexp_replace,concat,lit,split
from pyspark.ml.feature import Tokenizer, StopWordsRemover
import nltk
import string
import re
from nltk.stem.snowball import SnowballStemmer 
from pyspark.sql.functions import udf
import pyspark.sql.types as T
import pyspark.sql.functions as F
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS
from itertools import filterfalse
from nltk.corpus import wordnet as wn

In [17]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
!wget -q !wget  https://www.dropbox.com/s/cn2utnr5ipathhh/all-the-news-2-1.zip?dl=0

In [22]:
! unzip /content/all-the-news-2-1.zip?dl=0.1

Archive:  /content/all-the-news-2-1.zip?dl=0.1
  inflating: all-the-news-2-1.csv    


In [23]:
data= spark.read.csv("/content/all-the-news-2-1.csv",header=True,inferSchema=True)

In [45]:
new_data=data.select(data.columns[7:])

In [46]:
from pyspark.sql.functions import col,split,udf

In [47]:
final_data=new_data.drop(col("url"))

In [48]:
final_data.show()

+--------------------+--------------------+--------------------+--------------------+
|               title|             article|             section|         publication|
+--------------------+--------------------+--------------------+--------------------+
|We should take co...|"This post is par...| several critics ...|         for example|
|Colts GM Ryan Gri...| The Indianapolis...|                null|                null|
|                null|                null|                null|                null|
|Trump denies repo...|DAVOS, Switzerlan...|               Davos|             Reuters|
|France's Sarkozy ...|PARIS (Reuters) -...|          World News|             Reuters|
|Paris Hilton: Wom...|"Paris Hilton arr...|                null|                 TMZ|
|ECB's Coeure: If ...|BERLIN, June 17 (...|                null|                null|
|                null|                null|                null|                null|
|Venezuela detains...|CARACAS (Reuters)...|          W

In [49]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, lower, regexp_replace
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from nltk.stem.snowball import SnowballStemmer

In [50]:
from pyspark.sql.functions import lower,col,regexp_extract,filter,explode

In [51]:
import pyspark.sql.types as T
import pyspark.sql.functions as F 
from pyspark.sql.functions import split


In [52]:
def clean_text(c): ##cleaning text using clean_text function
  c = lower(c)
  c = regexp_replace(c, "^rt ", "")
  c = regexp_replace(c, "(https?\://)\S+", "")
  c = regexp_replace(c, "[^a-zA-Z0-9\\s]", "")
  return c

clean_text_df = final_data.select(clean_text(col("article")).alias("text"))

clean_text_df.printSchema()
clean_text_df.show(10)

root
 |-- text: string (nullable = true)

+--------------------+
|                text|
+--------------------+
|this post is part...|
| the indianapolis...|
|                null|
|davos switzerland...|
|paris reuters  fo...|
|paris hilton arri...|
|berlin june 17 re...|
|                null|
|caracas reuters  ...|
|if only every day...|
+--------------------+
only showing top 10 rows



In [53]:
@udf(returnType=T.ArrayType(T.StringType())) #create UDF function for tokenizeing data
def word_tokenize(x):
  if x == None:
    return ""
  else:
    return nltk.word_tokenize(x)

In [54]:
tokenize_data=clean_text_df.select(word_tokenize(F.col("text")).alias("text"))

In [55]:
tokenize_data.show()

+--------------------+
|                text|
+--------------------+
|[this, post, is, ...|
|[the, indianapoli...|
|                null|
|[davos, switzerla...|
|[paris, reuters, ...|
|[paris, hilton, a...|
|[berlin, june, 17...|
|                null|
|[caracas, reuters...|
|[if, only, every,...|
|[google, io, the,...|
|[china, is, dismi...|
|[elizabeth, warre...|
|[reuters, the, su...|
|[joakim, noahs, m...|
|[jermaine, jackso...|
|[london, reuters,...|
|[nancy, pelosi, i...|
|[the, nonpartisan...|
|[the, threat, of,...|
+--------------------+
only showing top 20 rows



In [56]:
tokens = tokenize_data.select(explode(col("text")).alias("tokens"))

In [57]:
tokens.show()

+-----------+
|     tokens|
+-----------+
|       this|
|       post|
|         is|
|       part|
|         of|
|  polyarchy|
|         an|
|independent|
|       blog|
|   produced|
|         by|
|        the|
|  political|
|     reform|
|    program|
|         at|
|        new|
|    america|
|          a|
| washington|
+-----------+
only showing top 20 rows



In [58]:
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol="tokens", outputCol="vector")
vector_df = tokenizer.transform(tokens).select("vector")

vector_df.printSchema()
vector_df.show(10)

root
 |-- vector: array (nullable = true)
 |    |-- element: string (containsNull = true)

+-------------+
|       vector|
+-------------+
|       [this]|
|       [post]|
|         [is]|
|       [part]|
|         [of]|
|  [polyarchy]|
|         [an]|
|[independent]|
|       [blog]|
|   [produced]|
+-------------+
only showing top 10 rows



In [59]:
remover = StopWordsRemover()
stopwords = remover.getStopWords() 

# Display default list
stopwords[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your']

In [60]:
# Specify input/output columns
remover.setInputCol("vector")
remover.setOutputCol("vector_no_stopw")

# Transform existing dataframe with the StopWordsRemover
vector_no_stopw_df = remover.transform(vector_df).select("vector_no_stopw")

# Display
vector_no_stopw_df.printSchema()
vector_no_stopw_df.show()

root
 |-- vector_no_stopw: array (nullable = true)
 |    |-- element: string (containsNull = true)

+---------------+
|vector_no_stopw|
+---------------+
|             []|
|         [post]|
|             []|
|         [part]|
|             []|
|    [polyarchy]|
|             []|
|  [independent]|
|         [blog]|
|     [produced]|
|             []|
|             []|
|    [political]|
|       [reform]|
|      [program]|
|             []|
|          [new]|
|      [america]|
|             []|
|   [washington]|
+---------------+
only showing top 20 rows



In [61]:
# Import stemmer library
from nltk.stem.porter import *
stemmer = PorterStemmer()

In [62]:
# Create stemmer python function
def stem(in_vec):
    out_vec = []
    for t in in_vec:
        t_stem = stemmer.stem(t)
        if len(t_stem) > 2:
            out_vec.append(t_stem)       
    return out_vec

In [63]:
# Create user defined function for stemming with return type Array<String>
from pyspark.sql.types import *
stemmer_udf = udf(lambda x: stem(x), ArrayType(StringType()))

In [64]:
# Create new df with vectors containing the stemmed tokens 
vector_stemmed_df = (
    vector_no_stopw_df
        .withColumn("vector_stemmed", stemmer_udf("vector_no_stopw"))
        .select("vector_stemmed")
  )

# Rename df and column for clarity
production_df = vector_stemmed_df.select(col("vector_stemmed").alias("bow"))

In [65]:
# Display
production_df.printSchema()
production_df.show()

root
 |-- bow: array (nullable = true)
 |    |-- element: string (containsNull = true)

+------------+
|         bow|
+------------+
|          []|
|      [post]|
|          []|
|      [part]|
|          []|
| [polyarchi]|
|          []|
|  [independ]|
|      [blog]|
|    [produc]|
|          []|
|          []|
|     [polit]|
|    [reform]|
|   [program]|
|          []|
|       [new]|
|   [america]|
|          []|
|[washington]|
+------------+
only showing top 20 rows



In [73]:
token_groups = production_df.groupby(col("bow"))

In [74]:
token_groups

In [75]:
token_count =token_groups.count()

In [76]:
BOW_vector=token_count.orderBy("count",ascending=False)

In [77]:
BOW=BOW_vector.show(50)  ##create Bag Of Word

+----------+---------+
|       bow|    count|
+----------+---------+
|        []|393234370|
|    [said]|  4931937|
|    [year]|  2436739|
|     [new]|  2276717|
|     [one]|  2099003|
|   [trump]|  2013837|
|    [like]|  1917987|
|  [report]|  1781822|
|    [time]|  1726524|
|   [peopl]|  1701317|
|    [also]|  1700307|
| [compani]|  1687898|
|   [state]|  1581131|
|     [say]|  1517079|
|  [presid]|  1334749|
|   [first]|  1288641|
| [percent]|  1278291|
|    [make]|  1275585|
|    [last]|  1264809|
|    [work]|  1231943|
|     [two]|  1211942|
|     [get]|  1194458|
|     [use]|  1133500|
| [million]|  1127297|
|  [reuter]|  1111078|
|     [day]|  1000896|
|  [includ]|   996050|
|    [even]|   940189|
|    [week]|   938312|
|    [take]|   936473|
|  [govern]|   913370|
|    [show]|   912158|
|    [want]|   879192|
|    [call]|   875449|
|    [back]|   864204|
|    [hous]|   858504|
|     [may]|   855749|
| [countri]|   841240|
|   [month]|   839748|
|    [come]|   837678|
|  [market]